# HCT Survival Predictions

Goal:  Develop models to improve the prediction of transplant survival rates for patients undergoing allogeneic Hematopoietic Cell Transplantation (HCT) — an important step in ensuring that every patient has a fair chance at a successful outcome, regardless of their background.

Improving survival predictions for allogeneic HCT patients is a vital healthcare challenge. Current predictive models often fall short in addressing disparities related to socioeconomic status, race, and geography. Addressing these gaps is crucial for enhancing patient care, optimizing resource utilization, and rebuilding trust in the healthcare system.

The goal is to address disparities by bridging diverse data sources, refining algorithms, and reducing biases to ensure equitable outcomes for patients across diverse race groups. Your work will help create a more just and effective healthcare environment, ensuring every patient receives the care they deserve.

Dataset Description

The dataset consists of 59 variables related to hematopoietic stem cell transplantation (HSCT), encompassing a range of demographic and medical characteristics of both recipients and donors, such as age, sex, ethnicity, disease status, and treatment details. The primary outcome of interest is event-free survival, represented by the variable efs, while the time to event-free survival is captured by the variable efs_time. These two variables together encode the target for a censored time-to-event analysis. The data, which features equal representation across recipient racial categories including White, Asian, African-American, Native American, Pacific Islander, and More than One Race, was synthetically generated using the data generator from synthcity, trained on a large cohort of real CIBMTR data.


    train.csv - the training set, with target efs (Event-free survival)
    test.csv - the test set; your task is to predict the value of efs for this data
    sample_submission.csv - a sample submission file in the correct format with all predictions set to 0.50
    data_dictionary.csv - a list of all features and targets used in dataset and their descriptions


## Import Package

In [1]:
import numpy as np
import pandas as pd

## Import Dataset

In [2]:
path_data_dictionary = "C:/Users/julia/Desktop/Yanjun/hct competition/data_dictionary.csv"
path_test = "C:/Users/julia/Desktop/Yanjun/hct competition/test.csv"
path_train = "C:/Users/julia/Desktop/Yanjun/hct competition/train.csv"
data_dictionary = pd.read_csv(path_data_dictionary)
path_submission = "C:/Users/julia/Desktop/Yanjun/hct competition/sample_submission.csv"
test = pd.read_csv(path_test)
train = pd.read_csv(path_train)
submission = pd.read_csv(path_submission)

In [3]:
data_dictionary

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN
5,hla_high_res_8,Recipient / 1st donor allele-level (high resol...,Numerical,NaN
6,tbi_status,TBI,Categorical,"['No TBI' 'TBI + Cy +- Other' 'TBI +- Other, <..."
7,arrhythmia,Arrhythmia,Categorical,['No' nan 'Yes' 'Not done']
8,hla_low_res_6,Recipient / 1st donor antigen-level (low resol...,Numerical,NaN
9,graft_type,Graft type,Categorical,['Peripheral blood' 'Bone marrow']


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      28800 non-null  int64  
 1   dri_score               28646 non-null  object 
 2   psych_disturb           26738 non-null  object 
 3   cyto_score              20732 non-null  object 
 4   diabetes                26681 non-null  object 
 5   hla_match_c_high        24180 non-null  float64
 6   hla_high_res_8          22971 non-null  float64
 7   tbi_status              28800 non-null  object 
 8   arrhythmia              26598 non-null  object 
 9   hla_low_res_6           25530 non-null  float64
 10  graft_type              28800 non-null  object 
 11  vent_hist               28541 non-null  object 
 12  renal_issue             26885 non-null  object 
 13  pulm_severe             26665 non-null  object 
 14  prim_disease_hct        28800 non-null

In [5]:
train.columns

Index(['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
       'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
       'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue',
       'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
       'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6',
       'hla_match_c_low', 'rituximab', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
       'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct',
       'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe',
       'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer',
       'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue',
       'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score',
       'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related',
       'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_hi

In [6]:
# remove Id from train
train = train.drop(columns = 'ID')
train1 = train.copy()

# manually delete train and data_dictionary, path_data_dictionary, path_train, path_test
del data_dictionary, path_data_dictionary, path_train, path_test
import gc
gc.collect()


0

In [7]:
train1.efs.value_counts('percent')

# efs = 1 means that this patient has Event after transplant surgery
# efs = 0 means that this patient doesn't has Event after transplant surgery

efs
1.0    0.539306
0.0    0.460694
Name: proportion, dtype: float64

In [8]:
train1.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


In [9]:
test

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


## Analyse of the Data

### Check the proposition of missing data

In [10]:
missing = np.round(train1.isna().sum()/len(train1), 3) * 100
df_missing = pd.DataFrame(missing, columns=['values']).sort_values(by = 'values', ascending = True)
df_missing

,values
efs_time,0.0
race_group,0.0
age_at_hct,0.0
efs,0.0
year_hct,0.0
tbi_status,0.0
prod_type,0.0
prim_disease_hct,0.0
graft_type,0.0
donor_related,0.5


In [11]:
# mark different variables which has different category of missing data percentage:

# function to differentiate different category percentage of missing data
def color_map(percent):
  cmap = []
  for x in percent:
    if x >= 20:
      temp = 'background-color: red'
    elif x >= 5:
      temp = 'background-color: orange'
    elif x >= 1:
      temp = 'background-color: yellow'
    else:
      temp = 'background-color: green'
    cmap.append(temp)
  return cmap
# df_missing.style.map(color_map)
df_missing.style.apply(lambda x: color_map(df_missing['values']), axis = 0)

,values
efs_time,0.000000
race_group,0.000000
age_at_hct,0.000000
efs,0.000000
year_hct,0.000000
tbi_status,0.000000
prod_type,0.000000
prim_disease_hct,0.000000
graft_type,0.000000
donor_related,0.500000


### 1.process missing data: remove missing data in row whose missing data is less than 1%; 

In [12]:
# 1 find variables which has percentage of missing data less than 5%
indexless1 = np.where(missing < 1)
missingless1 = missing.index[indexless1]
print('variables which has percentage of missing data less than 1% :', missingless1)


variables which has percentage of missing data less than 1% : Index(['dri_score', 'tbi_status', 'graft_type', 'vent_hist',
       'prim_disease_hct', 'prod_type', 'year_hct', 'in_vivo_tcd',
       'age_at_hct', 'gvhd_proph', 'sex_match', 'race_group', 'donor_related',
       'efs', 'efs_time'],
      dtype='object')


In [13]:
len(train1)

28800

In [14]:
# delete missing data in variables which has less than 1 %, this is because missing data which is less than 1% almost has no influence on results.
# misssing_less5_object = train1.loc[:, missingless5].select_dtypes(include = 'object').columns
train1 = train1.dropna(subset= missingless1)

# delete missingless5 and indexless5, 
del missingless1, indexless1, missing, df_missing
gc.collect()
len(train1)

27559

In [15]:
# check the new dataset missing data 
missing = (train1.isnull().mean() * 100).sort_values(ascending = True)

### 2 Create a missing indicater to show wether variables are missing or not; 3 imputing missing numerical missing data with mean or fill missing category variables with Missing. 

In [16]:
from pandas import DataFrame


missing_variables = missing[missing >1].index
# first select missing data which is category
missing_category = train1[missing_variables].select_dtypes(object).columns
for x in missing_category:
  train1[x + '_Missing'] = train1[x].isnull().astype(float)
# second select missing data which is numerical 
missing_numerical = train1[missing_variables].select_dtypes (exclude = object).columns
for x in missing_numerical:
  train1[x + '_Missing'] = train1[x].isnull().astype(float)
train1
# fill missing category with missing and fill missing numerical with mean 
train1[missing_category] = train1[missing_category].fillna('missing')
train1[missing_numerical] = train1[missing_numerical].fillna(train1[missing_numerical].mean())

# delete missing_variables, missing_category, missing_numerical and also missing 
del missing_variables, missing_category, missing_numerical, missing
gc.collect()

0

In [17]:
# use onehot code to change category variables to numerical 
train1 = pd.get_dummies(train1, drop_first= True, dtype = float)

In [18]:
# Use Cox Proportional Hazards Model and L1 to determine whether the missing data has an impact on survival
from lifelines import CoxPHFitter
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# first split the train1 into train and test dataset
traindata, testdata = train_test_split(train1, train_size = 0.7, stratify= train1['efs'])
# separte X and y 
traindata_X = traindata.drop(columns = ['efs', 'efs_time'])
traindata_y = traindata[['efs', 'efs_time']]
testdata_X = testdata.drop(columns = ['efs', 'efs_time'])
testdata_y = testdata[['efs', 'efs_time']]

In [19]:
del traindata, testdata

In [20]:
# standardized traindata_X and test_data_X
std = StandardScaler()
traindata_X = traindata_X.astype(float)
testdata_X = testdata_X.astype(float)
traindata_X.iloc[:,:] = std.fit_transform(traindata_X.iloc[:,:])
testdata_X.iloc[:,:] = std.fit_transform(testdata_X.iloc[:,:])

In [21]:
traindata_X

,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,...,donor_related_Related,donor_related_Unrelated,"melphalan_dose_N/A, Mel not given",melphalan_dose_missing,cardiac_Not done,cardiac_Yes,cardiac_missing,pulm_moderate_Not done,pulm_moderate_Yes,pulm_moderate_missing
1592,0.592272,0.797150,0.746324,0.804861,0.829436,0.645154,-0.147776,0.582337,0.65606,-1.973579,...,-1.139021,1.166085,0.647930,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146
19468,0.592272,0.797150,0.746324,0.804861,0.829436,0.645154,0.751360,0.582337,0.65606,0.568087,...,-1.139021,1.166085,-1.543377,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146
14527,0.003256,0.003608,0.003108,0.003423,0.004257,0.645154,0.005541,0.582337,0.65606,0.568087,...,0.877947,-0.857570,-1.543377,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,2.113757,-0.27146
18693,0.592272,0.797150,-1.014514,0.804861,0.829436,0.645154,0.751360,0.582337,0.65606,0.568087,...,0.877947,-0.857570,-1.543377,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,2.113757,-0.27146
27648,0.592272,0.797150,0.746324,0.804861,0.829436,0.645154,0.751360,0.582337,0.65606,0.568087,...,-1.139021,1.166085,0.647930,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14992,0.592272,0.797150,0.746324,0.804861,0.829436,0.645154,0.751360,0.582337,0.65606,0.568087,...,-1.139021,1.166085,-1.543377,-0.22276,-0.072908,4.294947,-0.301452,-0.075729,-0.473091,-0.27146
5702,0.592272,0.797150,0.746324,0.804861,0.829436,0.645154,0.751360,0.582337,0.65606,0.568087,...,0.877947,-0.857570,-1.543377,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146
17055,0.592272,0.797150,0.746324,0.804861,0.224451,-1.833718,0.751360,0.582337,0.65606,0.568087,...,-1.139021,1.166085,0.647930,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146
25155,0.592272,0.081032,0.746324,-0.106741,0.224451,0.645154,0.751360,0.582337,0.65606,0.568087,...,0.877947,-0.857570,0.647930,-0.22276,-0.072908,-0.232832,-0.301452,-0.075729,-0.473091,-0.27146


### 4 Apply L1 regulsrization with cox Proportionsl Hazards Model

In [22]:
# find important variables
lass_cv = LassoCV(cv = 20)
lass_cv.fit(X = traindata_X, y = traindata_y['efs_time'])
select_features = traindata_X.columns[lass_cv.coef_ != 0]
select_features

Index(['hla_match_c_high', 'hla_match_dqb1_high', 'hla_nmdp_6',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low',
       ...
       'tce_div_match_HvG non-permissive', 'tce_div_match_missing',
       'donor_related_Related', 'melphalan_dose_N/A, Mel not given',
       'melphalan_dose_missing', 'cardiac_Not done', 'cardiac_Yes',
       'cardiac_missing', 'pulm_moderate_Not done', 'pulm_moderate_Yes'],
      dtype='object', length=174)

In [23]:
# important variables data 
traindata_X_select_y = pd.concat([traindata_X[select_features], traindata_y], axis = 1)

In [24]:
# delete select_features
del select_features

In [25]:
gc.collect()
# use coxphfitter to train the important dataset
cox_model1 = CoxPHFitter(penalizer = 0.1)
cox_model1.fit(traindata_X_select_y, duration_col = 'efs_time', event_col = 'efs')
# print the summary of the summary of coxphfitter
summary = cox_model1.summary
# find variables which p values are less than 0.05, meaning this variables are significantly important
cond = summary['p'] <= 0.05
select_feature = summary[cond].index

In [26]:
# print the C_index for this Cox Porprotional Hazards 
cox_model1.concordance_index_

0.6761533985941475

In [27]:
del traindata_X_select_y, summary
gc.collect()

0

In [28]:
# new important variables 
cox_model = CoxPHFitter(penalizer= 0.1)
traindata_X_select_y = pd.concat([traindata_X[select_feature], traindata_y], axis = 1)
cox_model.fit(traindata_X_select_y, duration_col= 'efs_time', event_col= 'efs')
cox_model.concordance_index_

0.6723702129120437

In [29]:
del traindata_X_select_y, cox_model, cox_model1
gc.collect()

0

In [30]:
# use survival random forest to train the model 
import_traindata_X = traindata_X[select_feature]
del select_feature
gc.collect()

0

In [31]:
del train1, traindata_X
gc.collect()

0

In [32]:
# use survival random forest to train the model 
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
train_sur_object = Surv.from_arrays(traindata_y['efs'], traindata_y['efs_time'])
rsf = RandomSurvivalForest(n_estimators = 30, max_depth = 10, max_features = 'sqrt', random_state = 1)
rsf.fit(import_traindata_X, train_sur_object)
c_index = rsf.score(import_traindata_X, train_sur_object)
c_index

0.7126234661355753

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sksurv.metrics import concordance_index_censored
param_grid = {"max_depth":[15, 20], 
              "min_samples_split" :[5, 10],
              "max_features":['log2', 'sqrt'], 
              "bootstrap" :[True, False]
              }
rsf1 = RandomSurvivalForest(n_estimators= 30, random_state= 1)

def cindex_scorer(estimator, X, y):
  y_pred = estimator.predict_survival_function(X)
  return concordance_index_censored(y['efs'], y['efs_time'], y_pred)
grid_serach = GridSearchCV(estimator= rsf1, 
                           cv = 2,
                           param_grid= param_grid,
                           n_jobs = 1, 
                           verbose = 2)


In [34]:
grid_serach.fit(import_traindata_X, train_sur_object)

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] END bootstrap=True, max_depth=15, max_features=log2, min_samples_split=5; total time= 1.1min
[CV] END bootstrap=True, max_depth=15, max_features=log2, min_samples_split=5; total time= 1.1min
[CV] END bootstrap=True, max_depth=15, max_features=log2, min_samples_split=10; total time= 1.0min
[CV] END bootstrap=True, max_depth=15, max_features=log2, min_samples_split=10; total time= 1.0min
[CV] END bootstrap=True, max_depth=15, max_features=sqrt, min_samples_split=5; total time= 1.2min
[CV] END bootstrap=True, max_depth=15, max_features=sqrt, min_samples_split=5; total time= 1.2min
[CV] END bootstrap=True, max_depth=15, max_features=sqrt, min_samples_split=10; total time= 1.2min
[CV] END bootstrap=True, max_depth=15, max_features=sqrt, min_samples_split=10; total time= 1.2min
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_split=5; total time= 1.2min
[CV] END bootstrap=True, max_depth=20, max_features=l

MemoryError: could not allocate 929955840 bytes

In [35]:
grid_serach.best_params_

{'bootstrap': False,
 'max_depth': 15,
 'max_features': 'log2',
 'min_samples_split': 10}

In [36]:
grid_serach.best_estimator_

RandomSurvivalForest(bootstrap=False, max_depth=15, max_features='log2',
                     min_samples_split=10, n_estimators=30, random_state=1)

In [37]:
grid_serach.best_score_

0.661409473054853

In [37]:
# # Penalized Cox models, such as Ridge or Lasso regression, can help mitigate issues caused by collinearity and missing data. In lifelines, you can use the penalizer argument to apply regularization to the Cox model.
# cph = CoxPHFitter(penalizer=0.1)
# # use new data train2 (which is train1 remove missing20)
# train2 = train12.copy()
# train2 = train2.drop(columns = missing20)

# # replace category nan with missing, and replace numerical nan with median 
# object_columns = train2.select_dtypes(include = object).columns
# train2[object_columns]= train2.select_dtypes(include = object).fillna('Missing')

# numerical_columns = train2.select_dtypes(exclude = object).columns
# train2[numerical_columns] = train2[numerical_columns].fillna(train2[numerical_columns].median())

# # because Cox Proportional Hazards Model only cope with numerical variables, so change category to numerical(get_dummies)
# train2 = pd.get_dummies(train2, drop_first= True, dtype = int)
# # remove gvhd_proph_FK+- others(not MMF,MTX) this columns in train2, because this columns all have the same value
# train2 = train2.drop(columns = ['gvhd_proph_FK+- others(not MMF,MTX)'])

# # standardize train2, train2 chnage to train
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()
# train2 = train2.astype(float)
# train2.iloc[:,:] = std.fit_transform(train2.iloc[:,:])

In [38]:
# # use train2 to fit the model 
# cph.fit(train2, duration_col= 'efs_time', event_col= 'efs')
# cph.print_summary()
# # the result of the model is moderate , beacuse the C- index is 0.62, so it need to improve the data quality

###### group variables with missing data less than 5% and more than 5%

In [39]:
# # Use Cox Proportional Hazards Model to determine whether the missing data has an impact on survival
# from lifelines import CoxPHFitter
# # 1 find variables which has percentage of missing data more than 5%
# indexmore5 = np.where(missing>= 5)
# missingmore5 = missing.index[indexmore5]
# print('variables which has percentage of missing data more than 5%: ', missingmore5)

In [40]:
# traintemp = train1.copy()

In [41]:
# for variables which has more than 5% of missing data, create a missing indicator
# for col in missingmore5:
#   traintemp[col +'_missing'] = train1[col].isnull().astype(int)

In [42]:
# # Penalized Cox models, such as Ridge or Lasso regression, can help mitigate issues caused by collinearity and missing data. In lifelines, you can use the penalizer argument to apply regularization to the Cox model.
# cph = CoxPHFitter(penalizer=0.1)
# # use new data train2 (which is train1 remove missing20)
# train3 = traintemp.copy()

# # replace category nan with missing, and replace numerical nan with median 
# object_columns = train3.select_dtypes(include = object).columns
# train3[object_columns]= train3.select_dtypes(include = object).fillna('Missing')

# numerical_columns = train3.select_dtypes(exclude = object).columns
# train3[numerical_columns] = train3[numerical_columns].fillna(train3[numerical_columns].median())

# # because Cox Proportional Hazards Model only cope with numerical variables, so change category to numerical(get_dummies)
# train3 = pd.get_dummies(train3, drop_first= True, dtype = int)
# # remove gvhd_proph_FK+- others(not MMF,MTX) this columns in train3, because this columns all have the same value
# train3 = train3.drop(columns = ['gvhd_proph_FK+- others(not MMF,MTX)'])

# # standardize train3, train3 chnage to train
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()
# train3 = train3.astype(float)
# train3.iloc[:,:] = std.fit_transform(train3.iloc[:,:])

In [43]:
# # use train3 to fit the model 
# cph.fit(train3, duration_col= 'efs_time', event_col= 'efs')
# cph.print_summary()
# # the result of the model is moderate , beacuse the C- index is 0.63, so it need to improve the data quality

#### We know some variables with missing data are very important, so we are going to use random forests for survival to handle missing values directly

In [44]:
# import gc
# gc.collect()

# from sksurv.ensemble import RandomSurvivalForest
# from sksurv.util import Surv

# # split X and y from train4
# train4_X = train4.drop(columns= ['efs', 'efs_time']).astype(np.float32)
# train4_y = train4['efs_time']
# train4_r = train4['efs']

# # use Surv to create survival object
# surv_object = Surv.from_arrays( train4_r, train4_y)

# # delete train4
# del train4, train4_r, train4_y
# gc.collect()

# # create and fit the random survival forest model
# rsf = RandomSurvivalForest(n_estimators= 100, max_depth= 10, n_jobs = 1, max_features= 'sqrt', random_state= 1)
# rsf.fit(train4_X, surv_object)
# # after training the model, force garbage collection

# gc.collect()
# c_index = rsf.score(train4_X, surv_object)
# c_index

In [45]:
# test4_X = test4.drop(columns = ['efs', 'efs_time'])
# test4_y = test4['efs_time']
# test4_r = test4['efs']


# # calculate the C-index for test data
# y_test = Surv.from_arrays(test4_r, test4_y)

# # delete test4, test4_r, test4_y
# del test4, test4_r, test4_y
# gc.collect()

# c_test_index = rsf.score(test4_X, y_test)
# c_test_index

In [46]:
# from sklearn.inspection import permutation_importance

# # access the feature importance 
# # because train4_X is quite big data, use smaller data from train4_X
# small_train4_X = train4_X.sample(frac = 0.1 , random_state= 1)
# small_surv_object = surv_object[small_train4_X.index]
# feature_importances = permutation_importance(rsf, small_train4_X, small_surv_object, n_repeats= 2, n_jobs = 1 , random_state= 1)
# feature_importances

### There are negative permutation importance, whihc means that shuffle these variables improve the modelcompare the original data, this is maybe because the variables are irrelevant, noisy or correlated with other variables, Let check the feature correlation

In [47]:
# # delete small_train4_X, small_surv_object
# del small_train4_X, small_surv_object
# gc.collect()

# # give me any variables whose features_importances > 0 
# # feature_importancesbig0 = feature_importances.importances_mean > 0
# # mean_importancebig0 = feature_importances.importances_mean[feature_importancesbig0].mean()

# important_variables = [j for i, j in enumerate(train4_X.columns) if (feature_importances.importances_mean[i] > 0) ]

# # print(train4_X.columns[feature_importances.importances_mean > 0])
# important_variables


In [48]:
# # calculate VIF (Variance Inflation Factor) 
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools import add_constant

# # use train4_X(efs and efs_time are already removed) to calculate VIF


# # calculate VIF for each feature (because there are some missing data, so can't use VIF directly, so need to remove all missing data)


# train4_X_dropna = train4_X.dropna()

# indicater = 1

# while indicater:
#   vif = [variance_inflation_factor(exog = train4_X_dropna, exog_idx = i) for i in range(len(train4_X_dropna.columns))]
 
#   remain_features = train4_X_dropna.columns
#   if max(vif) > 5: 
#     index_max = np.where(vif == max(vif))
#     column_max = train4_X_dropna.columns[index_max]
#     train4_X_dropna = train4_X_dropna.drop(columns= column_max )
#   else: 
#     indicater = 0

# vif_feature = pd.DataFrame(data = {'remain_features': remain_features, 'vif' : vif})
# vif_feature

In [49]:
# # let check the multicolinarity of variables in the train4_X_dropna
# train5_X_dropna = train4_X.dropna()
# train5_X_dropna_const = add_constant(train5_X_dropna)
# vif = [variance_inflation_factor(exog = train5_X_dropna_const, exog_idx = i) for i in range(len(train5_X_dropna_const.columns))]
# vif_variables = pd.DataFrame(data = {'vaiables':train5_X_dropna_const.columns, 'vif': vif})
# vif_variables.head(149)

In [50]:
# # check whihc variables are deleted from train4_X.columns and also look at features_importance of variables which are depleted 
# for i, j in enumerate(train4_X.columns):
#   if i in remain_features:
#     continue
#   else:
#     print(f'{j} is deleted, and its features_importance is : {feature_importances.importances_mean[i]}')

In [51]:
# # create and fit the random survival forest model with new dataset

# rsf = RandomSurvivalForest(n_estimators= 100, max_depth= 10, n_jobs = 1, max_features= 'sqrt',random_state= 1)
# rsf.fit(train6_X, surv_object)

# # calculate the c-index for the new train dataset
# gc.collect()
# c_index = rsf.score(train6_X, surv_object)
# c_index

In [52]:
# # calculate the C-index for new test data
# test6_X = test4_X[important_variables]
# c_test_index = rsf.score(test6_X, y_test)
# c_test_index



### Use L1 regularization to a Cox Proportional Hazards

### Use random forests, XGBoost, or LightGBM to handle missing data, or Cox Proporttional Hazzrds model 